In [1]:
import pandas as pd 
from dash import Dash, html, Input, Output,callback, dcc
import dash 
casos = pd.read_csv('caso_full.csv', sep = ',')
import plotly.express as px
import dash_bootstrap_components as dbc


In [2]:
casos.head()

,city,city_ibge_code,date,epidemiological_week,estimated_population,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths
0,Rio Branco,1200401.0,2020-03-17,202012,413418.0,407319.0,False,False,3,0.72566,2020-03-17,0.0,0,1,city,AC,3,0
1,NaN,12.0,2020-03-17,202012,894470.0,881935.0,False,False,3,0.33539,2020-03-17,0.0,0,1,state,AC,3,0
2,Rio Branco,1200401.0,2020-03-18,202012,413418.0,407319.0,False,False,3,0.72566,2020-03-18,0.0,0,2,city,AC,0,0
3,NaN,12.0,2020-03-18,202012,894470.0,881935.0,False,False,3,0.33539,2020-03-18,0.0,0,2,state,AC,0,0
4,Rio Branco,1200401.0,2020-03-19,202012,413418.0,407319.0,False,False,4,0.96754,2020-03-19,0.0,0,3,city,AC,1,0


In [3]:
casos['data'] = pd.to_datetime(casos['date'])
casos['ano'] = casos['data'].dt.year
casos['mes'] = casos['data'].dt.month

In [4]:
casos = casos[['city', 'epidemiological_week', 'state', 'new_confirmed', 'new_deaths', 'ano','mes']]

In [5]:
casos.head()

,city,epidemiological_week,state,new_confirmed,new_deaths,ano,mes
0,Rio Branco,202012,AC,3,0,2020,3
1,NaN,202012,AC,3,0,2020,3
2,Rio Branco,202012,AC,0,0,2020,3
3,NaN,202012,AC,0,0,2020,3
4,Rio Branco,202012,AC,1,0,2020,3


In [6]:
casos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3853648 entries, 0 to 3853647
Data columns (total 7 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   city                  object
 1   epidemiological_week  int64 
 2   state                 object
 3   new_confirmed         int64 
 4   new_deaths            int64 
 5   ano                   int32 
 6   mes                   int32 
dtypes: int32(2), int64(3), object(2)
memory usage: 176.4+ MB


In [7]:
estados_regioes = {
    'AC': 'Norte',
    'AL': 'Nordeste',
    'AM': 'Norte',
    'AP': 'Norte',
    'BA': 'Nordeste',
    'CE': 'Nordeste',
    'DF': 'Centro-Oeste',
    'ES': 'Sudeste',
    'GO': 'Centro-Oeste',
    'MA': 'Nordeste',
    'MG': 'Sudeste',
    'MS': 'Centro-Oeste',
    'MT': 'Centro-Oeste',
    'PA': 'Norte',
    'PB': 'Nordeste',
    'PE': 'Nordeste',
    'PI': 'Nordeste',
    'PR': 'Sul',
    'RJ': 'Sudeste',
    'RN': 'Nordeste',
    'RO': 'Norte',
    'RR': 'Norte',
    'RS': 'Sul',
    'SC': 'Sul',
    'SE': 'Nordeste',
    'SP': 'Sudeste',
    'TO': 'Norte'
}


In [8]:
casos['regiao'] = casos['state'].map(estados_regioes)

In [9]:
casos.head()

,city,epidemiological_week,state,new_confirmed,new_deaths,ano,mes,regiao
0,Rio Branco,202012,AC,3,0,2020,3,Norte
1,NaN,202012,AC,3,0,2020,3,Norte
2,Rio Branco,202012,AC,0,0,2020,3,Norte
3,NaN,202012,AC,0,0,2020,3,Norte
4,Rio Branco,202012,AC,1,0,2020,3,Norte


In [10]:
total_ano = casos[['state','new_deaths','ano','new_confirmed', 'regiao']].groupby(['state','ano','regiao']).sum().reset_index()
total_mes = casos[['state','mes','new_deaths','new_confirmed','ano','regiao']].groupby(['state','mes','ano','regiao']).sum().reset_index()


In [21]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import dash_bootstrap_components as dbc  # Certifique-se de ter adicionado o import para o dbc

app = dash.Dash(__name__)

# Supondo que você já tenha os DataFrames total_ano e total_estado carregados.

app.layout = html.Div(style={'backgroundColor': '#06141B'}, className="container", children=[
    # Criando os elementos dos filtros

    dbc.Row([dbc.Col([
        # Criando o botão de filtro de anos
        dcc.Dropdown(id='filtro_ano',
                     options=[{'label': ano, 'value': ano} for ano in total_ano['ano'].unique()],
                     value=2022,
                     style={'color': '#CCD0CF', 'backgroundColor': '#11212D',  # Usando a paleta para o dropdown
                            'borderColor': '#4A5C6A'}
                     ),

        dcc.Dropdown(id='filtro_estados',
                     options=[{'label': estado, 'value': estado} for estado in total_ano['state'].unique()],
                     style={'color': '#CCD0CF', 'backgroundColor': '#11212D',  # Usando a paleta para o dropdown
                            'borderColor': '#4A5C6A'}
                     )

    ])]),


    dbc.Row([
            dbc.Col(dbc.Card([
                dbc.CardBody(html.H4(id='maior_caso', children='Maior Número de Casos')),
            ], style={'backgroundColor': '#1C1C1C', 'color': '#CCD0CF'}), width=3),

            dbc.Col(dbc.Card([ 
                dbc.CardBody(html.H4(id ='maio_morte', children = 'Maior Número de Mortes')),
            ], style = {'backgroundColor': '#1C1C1C', 'color': '#CCD0CF'}), width=3),

            dbc.Col(dbc.Card([
                dbc.CardBody(html.H4(id='mario_regiao_caos', children= 'Maior região com número de mortes')),
            ], style = {'backgroundColor': '#1C1C1C', 'color': '#CCD0CF'}), width=3),

            dbc.Col(dbc.Card([
                dbc.CardBody(html.H4(id = 'maior_regiao_mortes', children = 'Maior regiaão com Número de mortes')),
            ], style = {'backgroundColor': '#1C1C1C', 'color': '#CCD0CF'}), width=3),
            ]),
    
    # Criando os elementos dos gráficos
    dbc.Row([
        dbc.Col(dcc.Graph(id='grafico_calor'),
                width=6, style={'padding': '10px', 'color': '#9BA8AB'}),
        dbc.Col(dcc.Graph(id='grafico_hist'), width=6, style={'padding': '10px'}),
    ], style={'display': 'flex'}),

    dbc.Row([
        dbc.Col(dcc.Graph(id='grafico_barras'), width=6, style={'padding': '10px'}),
        dbc.Col(dcc.Graph(id='grafico_rosca'), width=6, style={'padding': '10px'})
    ], style={'display': 'flex'})

])


In [22]:

@app.callback(
    [Output('grafico_calor', 'figure'),
     Output('grafico_hist', 'figure'),
     Output('grafico_barras', 'figure'),
     Output('grafico_rosca', 'figure')],
    [Input('filtro_estados', 'value'),
     Input('filtro_ano', 'value')]
)

def atualiza_charts(estado, ano):
    # Filtrando os dados com base no estado e ano selecionados para o mapa
    estado_df = total_ano[(total_ano['ano'] == ano) & (total_ano['state'] == estado)] if estado else total_ano[total_ano['ano'] == ano]

    # Criando o mapa geografico do Brasil
    fig_mapa = px.choropleth(estado_df,
                             geojson='https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson',
                             locations='state',
                             featureidkey='properties.sigla',
                             color='new_confirmed',
                             hover_name='state',
                             hover_data=['new_confirmed', 'new_deaths'],
                             color_continuous_scale='Blues',
                             title="Mapa Coroplético dos Casos por Estado no Brasil")

    fig_mapa.update_geos(
        scope="south america",  # Limitar o escopo ao Brasil
        projection_type="mercator",  # Usar projeção Mercator
        center={"lat": -14.2350, "lon": -51.9253},  # Centralizar no Brasil
        lataxis_range=[-35, 5],  # Ajustar as latitudes para focar no Brasil
        lonaxis_range=[-75, -30],  # Ajustar as longitudes para focar no Brasil    
        landcolor="#11212D",  # Cor da terra/continentes (fundo dos estados)
        showocean=True,  # Mostrar oceanos
        oceancolor="#253745",  # Cor dos oceanos
        lakecolor="#253745"  # Cor dos lagos
    )

    fig_mapa.update_layout(
        paper_bgcolor='#06141B',
        font=dict(color='#CCD0CF')  ,
        plot_bgcolor = '#CCD0CF'

    )

    fig_mapa.update_xaxes(
        title_font=dict(size=18, color='#CCD0CF')
    )

    # Filtrando os dados apenas pelo ano selecionado para o gráfico de linhas
    filtro_ano = total_ano[total_ano['ano'] == ano]

    # Histograma dos casos por região e ano
    fig_hist = px.histogram(filtro_ano,
                            x='regiao',
                            y='new_deaths',
                            hover_name='new_confirmed',
                            color='state',
                            title = 'Comparação de crescimento de mortes por região.')

    fig_hist.update_layout(
        paper_bgcolor='#06141B',
        font=dict(color='#CCD0CF')  # Cor do texto para combinar com o fundo
    )

    # Criando o gráfico de barras por ano entre estados
    fig_barras = px.bar(data_frame=estado_df,
                        x='new_deaths',
                        y='state',
                        color='regiao',
                        title='Índices de mortes entre estados',
                        orientation='h')

    fig_barras.update_layout(
        paper_bgcolor='#06141B',
        font=dict(color='#CCD0CF')  # Cor do texto para combinar com o fundo
    )

    fig_pie = px.pie(
        filtro_ano,
        values='new_confirmed',
        names='regiao',
        color_discrete_sequence=px.colors.sequential.RdBu,
        labels={'Região': 'regiao'},
        title = 'Porcentagem de casos confirmados por região'

    )

    fig_pie.update_traces(textinfo='label+percent')
    fig_pie.update_layout(
        paper_bgcolor='#06141B',
        font=dict(color='#CCD0CF')  # Cor do texto para combinar com o fundo
    )

    return fig_mapa, fig_hist, fig_barras, fig_pie

if __name__ == '__main__':
    app.run_server(debug=True)
